In [1]:
# two types pf recommender system
#1 content based
#2 colaborative filtering based
#3 hybrid

In [2]:
import pandas as pd
import numpy as np

In [3]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [4]:
#merging teh data.

movies = movies.merge(credits, on = 'title')

In [5]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [6]:
# Feature filtering. 
# we will keep the features which are needed. like,
#genres
#movie_id
#keywords (are tags, which are pretty important)
#overview
#title
#cast
#crew

movies = movies[['movie_id', 'title', 'genres', 'overview', 'keywords', 'cast', 'crew']]

In [7]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4809 entries, 0 to 4808
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4809 non-null   int64 
 1   title     4809 non-null   object
 2   genres    4809 non-null   object
 3   overview  4806 non-null   object
 4   keywords  4809 non-null   object
 5   cast      4809 non-null   object
 6   crew      4809 non-null   object
dtypes: int64(1), object(6)
memory usage: 300.6+ KB


In [8]:
movies.isnull().sum()

movie_id    0
title       0
genres      0
overview    3
keywords    0
cast        0
crew        0
dtype: int64

In [9]:
movies.dropna(inplace=True)

In [10]:
movies.duplicated().sum()

0

In [11]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [12]:
# converting features into the required features.
# converting string of list of dictionaries into list of dict.
# for that we can use the 'ast' a module.

import ast

In [13]:
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L
        

In [14]:
#genres

movies['genres'] = movies['genres'].apply(convert)

In [15]:
movies.iloc[0].keywords

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [16]:
# keywords

movies['keywords'] = movies['keywords'].apply(convert)

In [17]:
movies.head(1)

,movie_id,title,genres,overview,keywords,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","In the 22nd century, a paraplegic Marine is di...","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [18]:
def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter+= 1
        else:
            break
    return L

In [19]:
#cast

movies['cast'] = movies['cast'].apply(convert3)

In [20]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
        
    return L

In [21]:
#crew

movies['crew'] = movies['crew'].apply(fetch_director)

In [22]:
# overview
# split function method basically slipts the string data into list. 

movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [23]:
movies.head(1)

,movie_id,title,genres,overview,keywords,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[In, the, 22nd, century,, a, paraplegic, Marin...","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]


In [24]:
# removing the spaces and creating one word, helps our model to understand the tags accurately.

movies['genres'] = movies['genres'].apply(lambda x : [i.replace (' ', '') for i in x])
movies['overview'] = movies['overview'].apply(lambda x : [i.replace (' ', '') for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x : [i.replace (' ', '') for i in x])
movies['cast'] = movies['cast'].apply(lambda x : [i.replace (' ', '') for i in x])
movies['crew'] = movies['crew'].apply(lambda x : [i.replace (' ', '') for i in x])

In [25]:
movies.head(1)

,movie_id,title,genres,overview,keywords,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction]","[In, the, 22nd, century,, a, paraplegic, Marin...","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]


In [26]:
movies['tags'] = movies['genres'] + movies['overview'] + movies['keywords'] + movies['cast'] + movies['crew']

In [27]:
movies.head(1)

,movie_id,title,genres,overview,keywords,cast,crew,tags
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction]","[In, the, 22nd, century,, a, paraplegic, Marin...","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, ScienceFiction, I..."


In [28]:
new_df = movies[['movie_id', 'title', 'tags']]

In [29]:
new_df.head(1)

,movie_id,title,tags
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction, I..."


In [30]:
new_df['tags'] = new_df['tags'].apply(lambda x: ' '.join(x))

C:\Users\abc\AppData\Local\Temp\ipykernel_6356\895399174.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: ' '.join(x))


In [31]:
new_df['tags'][0]

'Action Adventure Fantasy ScienceFiction In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'

In [32]:
# converting into lower case, (as a good practice.)

new_df['tags'] = new_df['tags'].apply (lambda x : x.lower())

C:\Users\abc\AppData\Local\Temp\ipykernel_6356\2438540388.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply (lambda x : x.lower())


In [33]:
# text vactoriztion.

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=5000, stop_words='english')

In [34]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [35]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [36]:
# calcuating the distance btw vectors. far the distance = low similarity in vectors.
# 2 types of distance
#   -eucladian distance - the distance between tip to tip of a vector.
#   -cosine distance - the angle between the arrow of a vector. (we are going to use cosine, just because the dimentionality space is large(5000).)


In [37]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
# this is the array of arrays where it contains the similarities between the first movie to other 5000 movies.

similarity[0]        

In [ ]:
def recommendation(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True, key= lambda x:x[1])[1:6]
    for i in movies_list:
        print(new_df.iloc[i[0]].title)
    

In [46]:
recommendation('Superman')

Superman II
Superman Returns
Superman III
Superman IV: The Quest for Peace
Man of Steel


In [47]:
import pickle

In [51]:
pickle.dump(new_df.to_dict(),open('movies_dict.pkl', 'wb'))

In [52]:
pickle.dump(similarity, open('similarity.pkl', 'wb'))